# TP n°4 - Feature selection
**Objectif:**

Ce TP vise à étudier quelques méthodes de sélection de variables (Filtrage) en utilisant la bibliothèque Scikit-learn.

[Documentation scikit-learn -> feature selection](https://scikit-learn.org/stable/modules/feature_selection.html)

Durant ce TP, on va aussi utiliser un modèle de réduction de variables (AFD).
Dans Scikit-learn, l'analyse factorielle discriminante (AFD) est mise en oeuvre dans la classe [LinearDiscriminantAnalysis](http://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html)

# Scikit-learn
Scikit-learn ou sklearn est une bibliothèque python libre conçue pour effectuer de l'apprentissage automatique.
Elle propose un set d'algorithmes de classification, régression et regroupement etc.

# Dataset
Pour illustrer le propos, on va utiliser le dataset `sales.csv` et nous allons ajouter des variables aléatoires qui représentent du bruit.


In [ ]:
# Chargement du dataset sales.csv


In [ ]:
# Afficher les dimension du dataset
# TODO

# Afficher les informations du dataset
# TODO

Pour l'exemple, on va supposer que la variable `division` (variable expliquée) contient les étiquettes des classes.

Pour référence, appliquons l'étape décisionnelle de l'analyse discriminante (comme modèle décisionnel) sur les données initiales et ensuite sur les données auxquelles les nouvelles variables aléatoires ont été ajoutées :

In [ ]:
# Préparation des subsets d'entrainement et de test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encoding des variables catégorielles
# TODO

# Ajout de 2 variables pour bruiter
np.random.seed(42)
# TODO


In [ ]:
X = df.drop(['level of education', 'education labels', 'division'], axis=1) # variables explicatives
y = df['education labels'] # variable expliquée
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state=42)
print(f"dimension du subset d'entrainement: {X_train.shape}")
print(f"dimension du subset de test: {X_test.shape}")

In [ ]:
X_train[0:2]

In [ ]:
# Création du modèle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model = LinearDiscriminantAnalysis()

# entrainement avec les données initiales
# TODO

# entrainement avec les données bruitées
# TODO


### Question
Comparer les performances du modèle entrainé sur les données initiales puis sur les données bruitées.


### Correction


# Filtrage
La famille de filtrage est appliquée sans faire appel à un modèle prédictif.
Elle permet de maximiser l'information mutuelle entre la variable d'entrée et celle de sortie.
Elle minimise la redondance entre les variables d'entrée.
NB: Une coopération sous optimale avec le modèle prédictif qui n'intervient pas dans la sélection.
Cette approche présente un coût inférieur à celui de l'approche Wrapper.

## Suppression de variables à faible variance
La suppression de variables à faible variance consiste à éliminer les variables dont la variance est inférieure à un seuil (par défaut la valeur du seuil est 0, sont donc éliminées les variables constantes).
[Voir plus](https://scikit-learn.org/stable/modules/feature_selection.html#removing-features-with-low-variance)

Proposer une suppression de variables en utilisant le code ci-dessous

In [ ]:
# Exemple de filtrage avec la suppression de variables à faible variance
from sklearn.feature_selection import VarianceThreshold
# TODO

In [ ]:
# affichage des variances des différentes variables
# TODO

In [ ]:
# Fixer un seuil de variance à 10 pour éliminer quelques variables
# TODO

In [ ]:
# affichage des variances des différentes variables
# TODO

In [ ]:
# Affichage des variables sélectionnées
# TODO

### Question
Commenter la variance des variables ajoutées (pour `threshold=10`)
A travers une autre combinaison de variables, proposer une 2ème classification via le même modèle étudié `from sklearn.discriminant_analysis import LinearDiscriminantAnalysis`.
Commenter le résultat obtenu.

### Correction


In [ ]:
model = LinearDiscriminantAnalysis()
# TODO

## Sélection univariée
La sélection univariées cherche à déterminer (à travers des tests statistiques comme le test de `Chi2`) dans quelle mesure chaque variable d'entrée "explique" la variable de sortie; les variables les moins explicatives individuellement sont éliminées.

Cette méthode élimine les variables pour lesquelles les valeurs de l'information mutuelle avec la variable de sortie sont les plus faibles (c'est à dire, qui « expliquent » le moins bien la variable de sortie). Nous avons l'intention de garder la moitié des variables et utilisons donc la fonction `SelectKBest` (d'autres sont disponibles, voir la documentation).

[Doc. sélection univariée](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection)
[Doc. information mutuelle pour la classification](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif)

### Question
Utilisez la sélection univariée pour sélectionner les meilleurs groupe de variables qui explique le mieux la variable expliquée.

### Correction

In [ ]:
# Exemple de filtrage avec la sélection univariée
from sklearn.feature_selection import SelectKBest, f_oneway
# TODO

### Question
Que constatez-vous par rapport aux variables ajoutées (bruit) ?

### Correction

### Question
Décrire la performance du modèle décisionnel. Comparer aux résultats obtenus par la méthode précédente.

### Correction

# Wrapper
La famille Wrapper coopère directement avec le modèle prédictif.
Choix des variables qui maximisent les performances du modèle.
**Inconvénients :**
- Coût élevé
- Pas de justification théorique de la sélection des variables
- Incompréhension des relations de dépendances entre les variables.
- La procédure de sélection est spécifique au modèle utilisé.

## Sélection séquentielle
Un modèle décisionnel doit être développé sur chaque (sous-)ensemble candidat de variables d'entrée et c'est la performance du modèle qui caractérise le (sous-)ensemble de variable. Deux versions sont proposées, une incrémentale (*Forward-SFS*) et une décrémentale (*Backward-SFS*).
[Voir plus](https://scikit-learn.org/stable/modules/feature_selection.html#sequential-feature-selection)

## Elimination récursive de variables
**Principe:** apprentissage du modèle sur la totalité des variables d'entrée pour extraire la pertinence de chaque variable (variance).
[Voir plus](https://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination)

## Sélection avec SelectFromModel
**Principe:** apprentissage du modèle sur la totalité des variables d'entrée pour extraire la pertinence de chaque variable (variance).
[Voir plus](https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection-using-selectfrommodel)

# Embedding
L'embedding (ou l'intégration) est une opération de sélection qui est intégrée à la méthode de construction de modèle. Pas de sur-coût par rapport à la construction du modèle mais cette approche ne peut pas être utilisée avec tout type de modèle.


# Exercice
Quelles sont les meilleures caractéristiques du dataset `sales.csv` qui permettent de prédire au mieux le salaire ?
**Note:** Vous pouvez utiliser ce modèle: `from sklearn.linear_model import LinearRegression`


# TP suivant ?
TP n°6: La réduction des variables par réduction des dimensions --> ACP.
